In [1]:
%reload_ext Cython

In [ ]:
import training
import importlib
importlib.reload(training)
train_data = training.TrainingData_PQR("../extracted/good_games.txt",80)

In [ ]:
import cProfile
import tensorflow as tf
import os
import re
import chess
import test
import boardclass
#import training
import importlib
import numpy as np
importlib.reload(boardclass)
importlib.reload(test)
importlib.reload(training)

In [ ]:
board = boardclass.ChessBoard(-1)
board.train_model(80,train_data,1)


board.restore_model()

score,move = board.return_best_move(1)
###board.close_session()
##
print(score,move)
#tf.reset_default_graph()

In [ ]:
for i in range(20):
    score,move = board.return_best_move(1)
    print("Score: {}".format(score))
    board.make_move(move)
    board.show_board()

In [ ]:

importlib.reload(test)
%run testcases.py